# Recommender System
## Group 7

This shows some attempts to build a recommender system

In [1]:
# import libraries
import graphlab as gl
import numpy as np
import pandas as pd

## Input data : keep a dataframe containing user-item interactions

In [2]:
# create SFrame for London twitter data
london_sf = gl.SFrame.read_csv('../DataSources/FourCityDataset/twitter/london.txt',
                        delimiter=' ',header=False)
london_sf.rename({'X1':'user_id','X2':'venue_id','X3':'#visits(normalized)'})

This non-commercial license of GraphLab Create for academic use is assigned to Y.Li-25@student.tudelft.nl and will expire on February 22, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\xps\AppData\Local\Temp\graphlab_server_1490890877.log.0


Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\DataSources\FourCityDataset\twitter\london.txt

Parsing completed. Parsed 100 lines in 0.078463 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\DataSources\FourCityDataset\twitter\london.txt

Parsing completed. Parsed 48852 lines in 0.068012 secs.

user_id,venue_id,#visits(normalized)
8493742,4ac518c5f964a520e1a420e3,0.666666666667
28777828,4c2ceb692219c92835e5a648,0.666666666667
162511952,4babef9df964a52012d73ae3,0.75
18816105,4aeb31a3f964a5207bbf21e3,0.666666666667
18319254,4af7074ff964a520fa0422e3,0.666666666667
376938788,4b7c1c76f964a520917d2fe3,0.8
20669549,4ac518e7f964a52053ab20e3,0.666666666667
20735661,4b838e75f964a520a30931e3,0.75
138078717,4f5a594fe4b0b04917548235,0.666666666667
139888796,4c177dffd4ee9521ba37ba05,0.666666666667


## Clean interactions, remove places contain "home"

In [3]:
london_sf = london_sf[london_sf['venue_id'].apply(lambda x: x.find('home'))==-1]

In [4]:
len(london_sf['venue_id'].unique())

11976

## Check unique users and items

In [5]:
print 'Unique users:',len(london_sf['user_id'].unique()) 
print 'Unique venues:',len(london_sf['venue_id'].unique())
print 'Total interactions:', len(london_sf)

Unique users: 9305
Unique venues: 11976
Total interactions: 46711


In [301]:
#category_list = london_sf_joined['category'].unique()
#delete_list = ['Fire Station','Physical Therapist',
#               'Animal Shelter','Housing Development',"Doctor's Office",
#              'Optical','Recruiting Agency','Real Estate',"Dentist's Office",
#             'Medical','Lawyer','Residential','Courthouse','Mental Health',
#              'Police Station','Prison','Administrative Building','Parking',
#             'Bus Station','Bus Stop','Bus','Office','Medical Lab','Emergency Room']

In [302]:
#london_sf_joined = london_sf_joined[london_sf_joined['category']
#                                    .apply(lambda x: x not in delete_list)]

## Add interaction data from Instagram user
still in question. Should we add it or not?

## Keep another list for item metadata
todo: enrich the metadata from foursquare

In [7]:
#items = gl.SFrame.read_csv(...)
import json
file_path = './cleaned_london_venues_data_twitter.json'
json_file = open(file_path)
jsonObj = json.load(json_file)
df = pd.DataFrame.from_dict(jsonObj,orient='index')
df['venue_id'] = df.index
sf = gl.SFrame(df)
sf_cleaned = sf['venue_id','name','category','rating','likes','checkins','tags','photo']
item_data = sf_cleaned
item_data = item_data['venue_id','category','rating','likes','checkins']
item_data_temp =item_data.dropna()
mean = item_data_temp['rating'].mean()
item_data=item_data.fillna('rating',mean)
item_data=item_data.fillna('likes',0)
item_data=item_data.fillna('checkins',0)
item_data.head()

venue_id,category,rating,likes,checkins
4a5f9446f964a520e0bf1fe3,Burgers,8.3,150.0,2419
4abcec53f964a520b98720e3,Hotel,9.0,32.0,1510
4abe4502f964a520558c20e3,Palace,9.0,5469.0,92542
4ac3ba25f964a520919c20e3,Hotel,7.2,4.0,303
4ac51183f964a52046a020e3,Portuguese,8.2,31.0,365
4ac51183f964a52048a020e3,Zoo,8.9,823.0,14252
4ac51183f964a52049a020e3,Art Gallery,9.5,1281.0,17161
4ac518b4f964a52067a020e3,Hotel,8.3,60.0,1893
4ac518b4f964a52071a020e3,Hotel,9.2,313.0,11020
4ac518b4f964a52073a020e3,Hotel,8.9,30.0,596


In [34]:
#sf_cleaned[sf_cleaned['photo']==[]]
def first_url(url_list):
    if len(url_list)==0:
        return ''
    else:
        return url_list[0]
sf_cleaned['photo'] = sf_cleaned['photo'].apply(lambda x: first_url(x))

In [12]:
london_sf_joined = london_sf.join(sf['venue_id','name','category'],on='venue_id')

# Keep a table for user infomation

In [13]:
user_data = gl.SFrame.read_csv('../DataSources/FourCityDataset/twitter/twitter_profiles.csv')

Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\DataSources\FourCityDataset\twitter\twitter_profiles.csv

Parsing completed. Parsed 100 lines in 0.659663 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,str,str,str,str,float,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,float,float,str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file E:\DST-3rd\IN4325 Information Retrieval\RecommenderSys\IR_RecommendationHE\DataSources\FourCityDataset\twitter\twitter_profiles.csv

Parsing completed. Parsed 21024 lines in 0.316432 secs.

In [14]:
user_data = user_data['id','g_post_count','g_home_city','g_home_country','g_country_count','g_gender','g_age','l_city','uclass']
user_data = user_data[user_data['l_city']=='london']

In [15]:
user_data.rename({'id':'user_id'})
user_data=user_data['user_id','g_gender','g_age']

In [16]:
user_data=user_data[user_data['user_id'].apply(lambda x:x in set(london_sf_joined['user_id']))]

## Create a simple model
1. Random split data into training data and validation data
2. We have a target column: #visits, thus we can create a model to predict which new places a user would visit frequently.

In [56]:
# split data into training and testing
#train_data, test_data = gl.recommender.util.random_split_by_user(london_sf, 'user_id', 'venue_id')
train_data,test_data = london_sf_joined.random_split(.8,seed=0)
factorization_model = gl.recommender.create(train_data,
                                                        user_id='user_id',
                                                       item_id='venue_id',
                                                        target = "#visits(normalized)",
                                                        user_data=user_data,
                                                        item_data=item_data
                                                                     )
#ranking_factorization_model = gl.ranking_factorization_recommender.create(training_data, 'user_id', 'venue_id',
#                                                                        target='#visits(normalized)')
#similarity_model = gl.recommender.item_similarity_recommender.create(train_data,'user_id','venue_id',user_data=user_data,item_data=item_data)

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 37419 observations with 9283 users and 11976 items.

Data prepared in: 0.162356s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.000429224

Final training RMSE: 0.00872683

## Use the model created to recommend places to a user

In [57]:
gl.canvas.set_target('ipynb')
view =factorization_model.views.overview(validation_set=test_data,user_data=user_data,
                                         item_data=sf_cleaned,
                                         item_description_column='category',
                                         item_name_column='name',
                                         item_url_column='photo')
view.show()

recommendations finished on 1000/5647 queries. users per second: 398.992

recommendations finished on 2000/5647 queries. users per second: 396.62

recommendations finished on 3000/5647 queries. users per second: 391.008

recommendations finished on 4000/5647 queries. users per second: 388.771

recommendations finished on 5000/5647 queries. users per second: 389.616

recommendations finished on 1000/5647 queries. users per second: 14489.6

recommendations finished on 2000/5647 queries. users per second: 11818

recommendations finished on 3000/5647 queries. users per second: 11659.7

recommendations finished on 4000/5647 queries. users per second: 11649.6

recommendations finished on 5000/5647 queries. users per second: 11417.5

## create a popularity model as our baseline
This model just recommend based on popularity, no personalization

In [70]:
popularity_model = gl.popularity_recommender.create(train_data,
                                                    user_id='user_id',
                                                    item_id='venue_id',
                                                    target = "#visits(normalized)" )

Recsys training: model = popularity

Warning: Ignoring columns name, category;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 37419 observations with 9246 users and 10885 items.

Data prepared in: 0.069021s

37419 observations to process; with 10885 unique items.

In [58]:
model_performance = gl.compare(test_data, [factorization_model, popularity_model], user_sample=0.05)
gl.show_comparison(model_performance,[factorization_model,popularity_model])

compare_models: using 282 users to estimate model performance
PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    |        0.0        |        0.0        |
|   3    |  0.00118203309693 | 0.000886524822695 |
|   4    | 0.000886524822695 | 0.000886524822695 |
|   5    |  0.00212765957447 |  0.0056146572104  |
|   6    |  0.00177304964539 |  0.0056146572104  |
|   7    |  0.00151975683891 |  0.0056146572104  |
|   8    |  0.00132978723404 |  0.0056146572104  |
|   9    |  0.00118203309693 |  0.0056146572104  |
|   10   |  0.00141843971631 |  0.00916075650118 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+----------------+-------------+
|

## Make recommendations

In [71]:
recommend_table = factorization_model.recommend(k=3)

recommendations finished on 1000/9283 queries. users per second: 379.209

recommendations finished on 2000/9283 queries. users per second: 382.38

recommendations finished on 3000/9283 queries. users per second: 390.872

recommendations finished on 4000/9283 queries. users per second: 394.989

recommendations finished on 5000/9283 queries. users per second: 398.007

recommendations finished on 6000/9283 queries. users per second: 399.858

recommendations finished on 7000/9283 queries. users per second: 401.018

recommendations finished on 8000/9283 queries. users per second: 402.216

recommendations finished on 9000/9283 queries. users per second: 394.774

In [101]:
recommend_table.head()

user_id,List of venue_id
225059732,"[4d89f0bdb799a35d7029a063, ..."
20063137,"[4ae811e5f964a5200cae21e3, ..."
1181033244,"[4ac518d2f964a5201fa720e3, ..."
10356152,"[4ec9222c29c22cc60e4e9e65, ..."
285211221,"[4bdd45a2645e0f47f3346b19, ..."
18157365,"[4add8098f964a520106521e3, ..."
808868670,"[4ac518d5f964a52013a820e3, ..."
236111793,"[53741abf498ebc92b703484a, ..."
21941712,"[4ac518d5f964a52013a820e3, ..."
332552379,"[4bdd45a2645e0f47f3346b19, ..."


In [106]:
import graphlab.aggregate as agg
#recommend_table=recommend_table.groupby(key_columns='user_id',operations=agg.CONCAT('venue_id'))
recommend_table.export_json('rec_by_records.json',orient='records')
#recommend_table.export_json('rec_by_rows.json', orient='rows')